<a href="https://colab.research.google.com/github/lsteffenel/ED-SNI-IntroDL/blob/main/06-Detect-Segment-YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Détection d'objets rapide avec YOLOv8

Même si on peut créer nos propres architectures DeepLearning, souvent il y a des bibliothèques et outils prêts pour des usages spécifiques.

La famille d'algorithmes YOLO est très connue par sa vitesse et précision pour des tâches de détection d'objets.

Dans les exemples ci-dessous, vous allez simplent utiliser YOLOv8 pour identifier des objets dans une image (libre à vous de modifier l'image source).

Deux modes seront utilisés :
- En mode détection, nous allons identifier la "zone" de l'objet, qui sera imprimé avec une boîte. C'est la façon la plus rapide car on ne fait qu'identifier l'objet et la zone dans lequel il est présent
- En mode segmentation sémantique, nous allons identifier chaque pixel appartenant à l'objet. Un masque de détection permet de connaître exactement où chaque objet se trouve, pas seulement sa zone.

Plusieurs autres exemples sont possibles avec YOLO (par exemple, détection de la pose d'une personne - grâce à une esquelette virtuelle) mais ça c'est pour un usage plus spécifique.


# Setup

installer la bibliothèque `ultralytics` avec `pip`, puis vérifier que l'installation s'est fait correctement.

In [ ]:
!pip install --quiet ultralytics
import ultralytics
ultralytics.checks()

## 1. Détection

Nous allons utiliser un modèle préexistant `yolov8n.pt`, déjà entraîné sur le dataset COCO.


In [ ]:
# Load YOLOv8n, train it on COCO128 for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # load a pretrained YOLOv8n detection model
model.train(data='coco8.yaml', epochs=3)  # train the model
results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image



In [ ]:
%matplotlib inline

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs

result.save('bus_detection.jpg')  # display to screen

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

img = np.asarray(Image.open('bus_detection.jpg'))
plt.imshow(img)
plt.show()

## 2. Ségmentation

Cette fois-ci, c'est un modèle de ségmentation `yolov8n-seg.pt` (aussi pré-entraîné avec COCO) qui sera utilisé.


In [ ]:
# Load YOLOv8n-seg, train it on COCO128-seg for 3 epochs and predict an image with it
from ultralytics import YOLO

modelseg = YOLO('yolov8n-seg.pt')  # load a pretrained YOLOv8n segmentation model
modelseg.train(data='coco8-seg.yaml', epochs=3)  # train the model
segresults = modelseg('https://ultralytics.com/images/bus.jpg')  # predict on an image


In [ ]:
import cv2
import random

# if you want all classes
yolo_classes = list(model.names.values())
classes_ids = [yolo_classes.index(clas) for clas in yolo_classes]

img_pil = Image.open('bus.jpg')
img = np.asarray(img_pil).copy() # Make a writable copy
# Convert from RGB (PIL default) to BGR (OpenCV default)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

colors = [random.choices(range(256), k=3) for _ in classes_ids]
for result in segresults:
    for mask, box in zip(result.masks.xy, result.boxes):
        points = np.int32([mask])
        color_number = classes_ids.index(int(box.cls[0]))
        cv2.fillPoly(img, points, colors[color_number])
cv2.imwrite("bus_segmentation.jpg",img)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

img = np.asarray(Image.open('bus_segmentation.jpg'))
plt.imshow(img)
plt.show()